# CentraleSupelec - Natural language processing
# Practical session n°7

## Natural Language Inferencing (NLI): 

(NLI) is a classical NLP (Natural Language Processing) problem that involves taking two sentences (the premise and the hypothesis ), and deciding how they are related (if the premise *entails* the hypothesis, *contradicts* it, or *neither*).

Ex: 


| Premise | Label | Hypothesis |
| --- | --- | --- |
| A man inspects the uniform of a figure in some East Asian country. | contradiction | The man is sleeping. |
| An older and younger man smiling. | neutral | Two men are smiling and laughing at the cats playing on the floor. |
| A soccer game with multiple males playing. | entailment | Some men are playing a sport. |

### Stanford NLI (SNLI) corpus

In this labwork, I propose to use the Stanford NLI (SNLI) corpus ( https://nlp.stanford.edu/projects/snli/ ), available in the *Datasets* library by Huggingface.

    from datasets import load_dataset
    snli = load_dataset("snli")
    #Removing sentence pairs with no label (-1)
    snli = snli.filter(lambda example: example['label'] != -1) 

## Subject

You are asked to provide an operational Jupyter notebook that performs the task of NLI. For that, you need to tackle the following aspects of the problem:

1. Loading and preprocessing the data
2. Designing a PyTorch model that, given two sentences, decides how they are related (*entails*, *contradicts* or *neither*.)
3. Training and evaluating the model using appropriate metrics
4. (Optional) Allowing to play with the model (forward user sentences and visualize the prediction easily)
5. (Optional) Providing visual insight about the model (i.e. visualizing the attention if your model is using attention)

Although it is not mandatory, I suggest that you use a transformer model to perform the task. For that, you can use the *Transformer* library by Huggingface.

## Evaluation

The evaluation will be based on several criteria:

- Clarity and readability of the notebook. The notebook is the report of you project. Make it easy and pleasant to read.
- Justification of implementation choices (i.e. the network, the cost funtion, the optimizer, ...)
- Quality of the code. The various deeplearning and NLP labworks provide many example of good practices for designing experiments with neural networks. Use them as inspirational examples!

## Additional recommendations

- You are not seeking to publish a research paper! I'm not expecting state-of-the-art results! The idea of this labwork is to assess that you have integrated the skills necessary to handle textual data using deep neural network techniques.

- This labwork will be evaluated but we are still here to help you! Don't hesitate to request our help if you are stuck.

- If you intend to use BERT based models, let me give you an advice. The bert-base-* models available in *Transformers* need more than 12Go to be fine-tuned on GPU. To avoid memory issues, you can use several solutions: 

    - Use a lighter BERT based model such as DistilBERT, ALBERT, ...
    - Train a classification model on top of BERT, whithout fine-tuning it (i.e. freezing BERT weights)

## Huggingface documentations

In case you want to use the huggingface *Datasets* and *Transformer* libraries (which I advice), here are some useful documentation pages:

- Dataset quick tour

    https://huggingface.co/docs/datasets/quicktour.html
    
- Documentation on data preprocessing for transformers

    https://huggingface.co/transformers/preprocessing.html
    
- Transformer Quick tour (with distilbert example for classification).

    https://huggingface.co/transformers/quicktour.html
    


In [1]:
import os 

os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [2]:
import os
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import pandas as pd
import numpy as np
import unicodedata
import re
import time
import random
import math

print(torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

1.9.1+cu111


## Loading and preprocessing the data

we will use distill bert model because it is lighter, therefore we use the distill bert tokenizer

In [3]:
from datasets import load_dataset

snli = load_dataset("snli")

#Removing sentence pairs with no label (-1)
snli = snli.filter(lambda example: example['label'] != -1) 

Reusing dataset snli (/usr/users/gpusdi1/gpusdi1_40/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_40/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-69916534c8046ad9.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_40/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-3eb05855f5b61765.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_40/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-48d9b7dcd4658c32.arrow


In [4]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased",num_labels=3)



def encode(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding='max_length')

dataset = snli.map(encode,batched=True)


Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_40/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-90e0b831d2a16088.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_40/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-ce9f5714ac7d5c80.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_40/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-79c6ce960916c330.arrow


In [5]:
print(dataset['test'][0])

{'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.', 'hypothesis': 'The church has cracks in the ceiling.', 'label': 1, 'input_ids': [101, 2023, 2277, 6596, 10955, 2000, 1996, 11678, 2004, 2027, 6170, 6569, 3560, 2774, 2013, 1996, 2338, 2012, 1037, 2277, 1012, 102, 1996, 2277, 2038, 15288, 1999, 1996, 5894, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

we make the column correspond to the inut of distillbert

In [6]:
dataset = dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_40/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-c217cc0c792eda1b.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_40/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-c69d3f35116b0780.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_40/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-59c026ccd4e335a3.arrow


In [7]:
import torch 

dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


train_dataset = dataset["train"]
test_dataset = dataset["test"]
validation_dataset = dataset["validation"]

batch_size = 10
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size)

example = next(iter(train_dataloader))

In [8]:
print(example)

# taper dans googlee freeziing bert model

{'input_ids': tensor([[ 101, 1037, 2711,  ...,    0,    0,    0],
        [ 101, 1037, 2711,  ...,    0,    0,    0],
        [ 101, 1037, 2711,  ...,    0,    0,    0],
        ...,
        [ 101, 1037, 2879,  ...,    0,    0,    0],
        [ 101, 1037, 2879,  ...,    0,    0,    0],
        [ 101, 2019, 3080,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 2, 0, 1, 0, 2, 2, 0, 1, 1])}


In [9]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification


model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

In [10]:
# for param in model.base_model.parameters():
#     param.requires_grad = False

In [11]:
from sklearn.metrics import accuracy_score

def metrics(loss, output_logits, labels):
    
    loss = loss.clone().detach()
    logits = output_logits.clone().detach()
    labels = labels.clone().detach()
    
    predictions = torch.argmax(F.softmax(logits, dim=-1), dim=1) # computing the prediction from the logits
    
    # tensors need to be on the cpu to convert to numpy
    loss = loss.cpu().numpy()
    labels = labels.cpu().numpy()
    predictions = predictions.cpu().numpy()
    
    loss = loss.item()
    acc = accuracy_score(labels, predictions)
    
    return loss, acc

In [ ]:
from transformers import get_linear_schedule_with_warmup
import tqdm as tqdm

device = 'cuda'
epochs = 3
warmup_steps = 4
train_steps = 2

model.to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, train_steps)

print('training the model')

for epoch in range(epochs):
    
    model.train()

    for batch in tqdm.tqdm(train_dataloader):
        
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # forward :
        outputs = model(**batch)
        
        loss = outputs.loss
        
        # back propagate :
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        print('metrics train')
        display_variables = metrics(loss, outputs.logits, batch['labels'])
        print(display_variables)
        

    
    model.eval()
    #metrics = {'loss': [], 'accuracy': [], 'f1': [],  'precision': [], 'recall': []}
    
    with torch.no_grad():
        for batch in tqdm(valid_dataloader):

            batch = {k: v.to(device) for k, v in batch.items()}

            # forward :
            outputs = model(**batch)

            loss = outputs.loss
        
    print('metrics validation')
    display_variables = metrics(loss, outputs.logits, batch['labels'])
    print(display_variables)
    
    

In [ ]:
from datasets import Dataset

def predict(hypothesis:str, premise:str): 
    input_dict = {'hypothesis': [hypothesis], 'premise': [premise]}
    dataset = Dataset.from_dict(input_dict)
    tokenized_dataset = dataset.map(encode, batched=True, batch_size=1)
    tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask']) # no label column
    dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size=1)
    
    model.eval()
    with torch.no_grad():
        # propagate only batch through model
        batch = next(iter(dataloader))
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        pred = torch.argmax(probs, dim=1)

        my_probs = probs.cpu().numpy()[0]
        my_pred = pred.cpu().numpy().item()

    idx_to_labels = {0: "entails", 1: "is neutral regarding", 2: "contradicts"}
    res = f"The premise ['{premise}'] {idx_to_labels[my_pred]} the hypothesis ['{hypothesis}']."
    
    return res, my_probs

In [ ]:
result, probs = predict("I am tall", "I have long legs")
print(result)
print(probs)